Mohammed Madi 20200386

## **Problem Statement**

A company has ten employees, seven machines, and thirty jobs to be completed. The time required 
to complete job 𝑗 by employee 𝑒 using machine 𝑚 is given and denoted by $𝑡_{𝑗𝑒𝑚}$. An employee can 
only perform one job at a time. Similarly, a machine can only execute one job at a time. A job 
cannot be divided among employees. Also, a job cannot be divided between machines. Develop a 
mathematical formulation that can be used to obtain the optimal job-employee-machine 
assignment if the company wants to minimize the total time spent by all machines to complete the 
seven jobs. Generate different scenarios and, using Cplex, implement and solve your formulation 
for these scenarios.

## **Sets**

$\mathcal{E}$: Set of employees that work at the company.

$\mathcal{M}$: Set of Machines that can be used.

$\mathcal{J}$: Set of Jobs to be completed.

## **Indices**

$e$: Index for an arbitrary element in $\mathcal{E}$.

$m$: Index for an arbitrary element in $\mathcal{M}$.

$j$: Index for an arbitrary element in $\mathcal{J}$

## **Data**

$t_{jem}, j \in \mathcal{J}, e \in \mathcal{E}, m \in \mathcal{M}$, The amount of time needed to completed job $j$ by employee $e$ using machine $m$.


## **Decision Variables**

$x_{jem}, j \in \mathcal{J}, e \in \mathcal{E}, m \in \mathcal{M}$, Decide which employee should do which job using which machine.

## **Linear Program**

**Objective Function**
\begin{align*}
\mathrm{minimize} \sum_{j \in \mathcal{J}}\sum_{e \in \mathcal{E}}\sum_{m \in \mathcal{M}} x_{jem}t_{jem}
\end{align*}

**S.T.**
\begin{gather}
\sum_{e \in \mathcal{E}} \sum_{m \in \mathcal{M}} x_{jem} = 1, \forall j \in \mathcal{J}, \text{ Each job can only be done by 1 employee and 1 machine.}\\
x_{jem} \in \{0, 1\}, \forall j \in \mathcal{J}, \forall e \in \mathcal{E}, \forall m \in \mathcal{M}, \text{Binary constraint.} \\
\end{gather}

## **Interpertation**

The assignment matrix is telling us which employee should use which machine to do the task, since we only care about total time, if the same employee gets multiple jobs he can perform them in any order he can.

I wanted to try to write the other objective function in which we care about time to finish, but I didnt have the time to do so.

(:

In [1]:
from docplex.mp.model import Model
model = Model(name = "HW1")

In [2]:
from random import randint
# Generating random data
num_employees = 10
num_machines = 7
num_jobs = 30

t = [[[randint(1, 100) for m in range(num_machines)] for e in range(num_employees)] for j in range(num_jobs)]



In [3]:
# Decision Variable
x = model.binary_var_cube(num_jobs, num_employees, num_machines, name = "x")

In [4]:
# Objective Function
model.minimize(model.sum(t[j][e][m] * x[(j, e, m)] for j in range(num_jobs) for e in range(num_employees) for m in range(num_machines)))

In [5]:
# Adding constraints
for j in range(num_jobs):
    model.add_constraint(model.sum(x[(j, e, m)] for e in range(num_employees) for m in range(num_machines)) == 1)

In [6]:
model.export_as_lp("HW1.lp")

'HW1.lp'

In [7]:
model.solve(log_output = True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 1206.000000 after 0.00 sec. (0.12 ticks)
Found incumbent of value 59.000000 after 0.00 sec. (0.18 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 30 rows and 2100 columns.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.71 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.98 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.98 ticks)


docplex.mp.solution.SolveSolution(obj=59,values={x_0_0_3:1,x_1_9_3:1,x_2..

In [8]:
# verifying the solution
obj_value = model.objective_value
assignment_matrix = [[[x[(j, e, m)].solution_value for j in range(num_jobs)] for e in range(num_employees)] for m in range(num_machines)]

sum_vals = 0
for e in range(num_employees):
    for j in range(num_jobs):
        for m in range(num_machines):
            if assignment_matrix[m][e][j] == 1:
                print(f"Employee {e} is assigned to Job {j} on Machine {m} with time {t[j][e][m]}")
                sum_vals += t[j][e][m]
                assert t[j][e][m] == min([t[j][e_][m_] for e_ in range(num_employees) for m_ in range(num_machines)])


assigned_jobs = sum(sum(sum(row) for row in matrix) for matrix in assignment_matrix)
assert assigned_jobs == num_jobs
print("Sum of all values in assignment matrix:", assigned_jobs)

assert sum_vals == obj_value
print("Min time:", obj_value)
print("Solution is correct!")

Employee 0 is assigned to Job 0 on Machine 3 with time 1
Employee 0 is assigned to Job 2 on Machine 6 with time 1
Employee 0 is assigned to Job 3 on Machine 3 with time 2
Employee 0 is assigned to Job 14 on Machine 0 with time 2
Employee 0 is assigned to Job 16 on Machine 0 with time 2
Employee 0 is assigned to Job 17 on Machine 4 with time 1
Employee 1 is assigned to Job 4 on Machine 5 with time 5
Employee 1 is assigned to Job 5 on Machine 2 with time 2
Employee 1 is assigned to Job 6 on Machine 5 with time 2
Employee 1 is assigned to Job 12 on Machine 2 with time 1
Employee 1 is assigned to Job 23 on Machine 4 with time 2
Employee 2 is assigned to Job 7 on Machine 0 with time 1
Employee 2 is assigned to Job 8 on Machine 3 with time 1
Employee 2 is assigned to Job 19 on Machine 1 with time 8
Employee 3 is assigned to Job 21 on Machine 5 with time 2
Employee 3 is assigned to Job 24 on Machine 0 with time 1
Employee 3 is assigned to Job 25 on Machine 2 with time 2
Employee 4 is assigned